In [55]:
# set up Python environment: numpy for numerical routines, and matplotlib for plotting
import numpy as np
import matplotlib.pyplot as plt
# display plots in this notebook
%matplotlib inline

In [56]:
# set display defaults
plt.rcParams['figure.figsize'] = (10, 10)        # large images
plt.rcParams['image.interpolation'] = 'nearest'  # don't interpolate: show square pixels
plt.rcParams['image.cmap'] = 'gray'  # use grayscale output rather than a (potentially misleading) color heatmap

In [57]:
import sys
caffe_root = '/Users/manoj/caffe/'  # this file should be run from {caffe_root}/examples (otherwise change this line)
sys.path.insert(0, caffe_root + 'python')

import caffe


In [58]:
import os
caffe.set_mode_cpu()


In [59]:
model_def = caffe_root + 'models/placesCNN_upgraded/places205CNN_deploy_upgraded.prototxt'
model_weights = caffe_root + 'models/placesCNN_upgraded/places205CNN_iter_300000_upgraded.caffemodel'


net = caffe.Net(model_def,      # defines the structure of the model
                model_weights,  # contains the trained weights
                caffe.TEST)     # use test mode (e.g., don't perform dropout)



In [60]:
IMAGE_WIDTH=227;
IMAGE_HEIGHT=227;


def transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT):


    #Histogram Equalization
    img[:, :, 0] = cv2.equalizeHist(img[:, :, 0])
    img[:, :, 1] = cv2.equalizeHist(img[:, :, 1])
    img[:, :, 2] = cv2.equalizeHist(img[:, :, 2])
     #Image Resizing
    img = cv2.resize(img, (img_width, img_height), interpolation = cv2.INTER_CUBIC)

    return img
def make_datum(img, label):


    return caffe_pb2.Datum(
        channels=3,
        width=IMAGE_WIDTH,
        height=IMAGE_HEIGHT,
        label=label,
        data=np.rollaxis(img, 2).tostring())

In [61]:
import sys
sys.path.append('/usr/local/lib/python2.7/site-packages')
import cv2

In [62]:
mean_blob = caffe.proto.caffe_pb2.BlobProto()

In [63]:
with open(caffe_root+'models/placesCNN_upgraded/places205CNN_mean.binaryproto') as f:
    mean_blob.ParseFromString(f.read())
mean_array = np.asarray(mean_blob.data, dtype=np.float32).reshape(
    (mean_blob.channels, mean_blob.height, mean_blob.width))

mean_array=mean_array.mean(1).mean(1)

In [64]:
transformer = caffe.io.Transformer({'data': net.blobs['data'].data.shape})
transformer.set_mean('data', mean_array)
#transformer.set_raw_scale('data',255)
transformer.set_transpose('data', (2,0,1))
#transformer.set_channel_swap('data', (2,1,0))  # swap channels from RGB to BGR


In [65]:
#extract feature vector for good exemplars
gb='good'
layer_list=['conv1','conv2','conv3','conv4','conv5','fc6','fc7','fc8']
#layer_list=['fc8']
test_input= '/Users/manoj/good_bad_placecnn/resized'+gb+'/'
text_file = open(test_input+gb+'_list.txt', "r")
lines = text_file.readlines()
labels_file = caffe_root + 'models/placesCNN_upgraded/categoryIndex_places205.csv'

for j in layer_list:
    featurefile= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+j+"_360.txt","w")
    for i in lines:  
        i = i.rstrip('\n')
        img = cv2.imread(test_input+i)
        img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
        net.blobs['data'].data[...] = transformer.preprocess('data', img)
        out = net.forward()
        out_prob = out['prob']
        label= np.loadtxt(labels_file, str, delimiter='\t')
        #exec("%s = %d" % (layer,2))  #convert string layer to variable layer
        layer=j
        nparr=net.blobs[layer].data[0].reshape(1,-1)  #numpy array of feature vector in the layer 
        nparr=nparr[0]
        list=nparr.tolist() #convert numpy array to list 
        strlist=map(str,list)
        string=' '.join(strlist)   # convert list to a single string 
        featurefile.write(string+"\n")
    print nparr.shape


(290400,)
(186624,)
(64896,)
(64896,)
(43264,)
(4096,)
(4096,)
(205,)


In [66]:
#extract feature vector for good exemplars
gb='bad'
layer_list=['conv1','conv2','conv3','conv4','conv5','fc6','fc7','fc8']
#layer_list=['fc8']
test_input= '/Users/manoj/good_bad_placecnn/resized'+gb+'/'
text_file = open(test_input+gb+'_list.txt', "r")
lines = text_file.readlines()
labels_file = caffe_root + 'models/placesCNN_upgraded/categoryIndex_places205.csv'

for j in layer_list:
    featurefile= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+j+"_360.txt","w")
    for i in lines:  
        i = i.rstrip('\n')
        img = cv2.imread(test_input+i)
        img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
        net.blobs['data'].data[...] = transformer.preprocess('data', img)
        out = net.forward()
        out_prob = out['prob']
        label= np.loadtxt(labels_file, str, delimiter='\t')
        #exec("%s = %d" % (layer,2))  #convert string layer to variable layer
        layer=j
        nparr=net.blobs[layer].data[0].reshape(1,-1)  #numpy array of feature vector in the layer 
        nparr=nparr[0]
        list=nparr.tolist() #convert numpy array to list 
        strlist=map(str,list)
        string=' '.join(strlist)   # convert list to a single string 
        featurefile.write(string+"\n")
    print nparr.shape



(290400,)
(186624,)
(64896,)
(64896,)
(43264,)
(4096,)
(4096,)
(205,)


In [76]:
#extract feature vector for good exemplars
gb='bad'
layer_list=['fc8']
#layer_list=['fc8']
test_input= '/Users/manoj/good_bad_placecnn/resized'+gb+'/'
text_file = open(test_input+gb+'_list.txt', "r")
lines = text_file.readlines()
labels_file = caffe_root + 'models/placesCNN_upgraded/categoryIndex_places205.csv'

for j in layer_list:
    featurefile= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+j+"_360.txt","w")
    for i in lines:  
        i = i.rstrip('\n')
        img = cv2.imread(test_input+i)
        img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
        net.blobs['data'].data[...] = transformer.preprocess('data', img)
        out = net.forward()
        out_prob = out['prob']
        label= np.loadtxt(labels_file, str, delimiter='\t')
        #exec("%s = %d" % (layer,2))  #convert string layer to variable layer
        layer=j
        nparr=net.blobs[layer].data[0].reshape(1,-1)  #numpy array of feature vector in the layer 
        nparr=nparr[0]
        list=nparr.tolist() #convert numpy array to list 
        strlist=map(str,list)
        string=' '.join(strlist)   # convert list to a single string 
        featurefile.write(string+"\n")
    print nparr.shape



(205,)


In [67]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.utils import shuffle

#cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

In [72]:
#six way classification 
#final version for 360 images 
gb='good'
layer_list=['conv1','conv2','conv3','conv4','conv5','fc6','fc7','fc8']
clf = svm.SVC(kernel='linear', C=2.03)   
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
#y_good=[1]*27+[2]*31+[3]*39+[4]*46+[5]*25+[6]*42
y_good=[1]*60+[2]*60+[3]*60+[4]*60+[5]*60+[6]*60
y_good=np.asarray(y_good)
#y_bad=[1]*34+[2]*39+[3]*32+[4]*31+[5]*42+[6]*32
y_good=[1]*60+[2]*60+[3]*60+[4]*60+[5]*60+[6]*60
y_bad=np.asarray(y_bad)
if gb=='good':
    y=y_good
    for layer in layer_list:
        list0= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+layer+"_360.txt", "r")
        X_good=[]
        for line in list0:
            #print line
            X_good.append(line.split())
        X_good=[[float(j) for j in i] for i in X_good]
        X_good=np.asarray(X_good)
        X=X_good
        score_good=cross_val_score(clf, X, y, cv=cv)
        print layer
        print score_good
        print score_good.mean()
else:
    y=y_bad
    for layer in layer_list:
        list0= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+layer+"_360.txt", "r")
        X_bad=[]
        for line in list0:
            X_bad.append(line.split())
        X_bad=[[float(j) for j in i] for i in X_bad]
        X_bad=np.asarray(X_bad)
        X=X_bad
        score_bad=cross_val_score(clf, X, y, cv=cv)
        print layer
        print score_bad
        print score_bad.mean()

conv1
[ 0.77777778  0.61111111  0.86111111  0.75        0.75        0.75
  0.77777778  0.61111111  0.63888889  0.83333333]
0.736111111111
conv2
[ 0.97222222  0.91666667  0.97222222  0.91666667  0.83333333  0.91666667
  0.94444444  0.97222222  0.86111111  0.97222222]
0.927777777778
conv3
[ 0.97222222  0.97222222  1.          0.97222222  0.91666667  0.97222222
  0.97222222  0.97222222  0.88888889  1.        ]
0.963888888889
conv4
[ 0.97222222  0.97222222  1.          0.97222222  0.97222222  0.97222222
  0.97222222  0.97222222  0.91666667  1.        ]
0.972222222222
conv5
[ 1.          0.97222222  1.          0.97222222  0.97222222  0.94444444
  0.94444444  1.          0.97222222  0.97222222]
0.975
fc6
[ 1.          1.          1.          0.97222222  1.          1.
  0.97222222  1.          1.          0.97222222]
0.991666666667
fc7
[ 1.          1.          1.          0.97222222  1.          1.
  0.97222222  1.          1.          1.        ]
0.994444444444
fc8
[ 1.          1.       

In [77]:
#extract feature vector for good exemplars
gb='bad'
layer_list=['fc8']
#layer_list=['fc8']
test_input= '/Users/manoj/good_bad_placecnn/resized'+gb+'/'
text_file = open(test_input+gb+'_list.txt', "r")
lines = text_file.readlines()
labels_file = caffe_root + 'models/placesCNN_upgraded/categoryIndex_places205.csv'

for j in layer_list:
    featurefile= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+j+"_360.txt","w")
    for i in lines:  
        i = i.rstrip('\n')
        img = cv2.imread(test_input+i)
        img = transform_img(img, img_width=IMAGE_WIDTH, img_height=IMAGE_HEIGHT)
        net.blobs['data'].data[...] = transformer.preprocess('data', img)
        out = net.forward()
        out_prob = out['prob']
        label= np.loadtxt(labels_file, str, delimiter='\t')
        #exec("%s = %d" % (layer,2))  #convert string layer to variable layer
        layer=j
        nparr=net.blobs[layer].data[0].reshape(1,-1)  #numpy array of feature vector in the layer 
        nparr=nparr[0]
        list=nparr.tolist() #convert numpy array to list 
        strlist=map(str,list)
        string=' '.join(strlist)   # convert list to a single string 
        featurefile.write(string+"\n")
    print nparr.shape



(205,)


In [25]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
from sklearn import svm
import numpy as np
from sklearn.model_selection import train_test_split

In [26]:
y=[1]*360+[0]*360
y=np.asarray(y)
y.shape
clf = svm.SVC(kernel='linear', C=2.03)
num_sample = 420
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)

In [ ]:
layer_list=['fc7']
for layer in layer_list:
    list1 = open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+layer+"_360.txt", "r")
    X_good=[]
    for line in list1 :
        X_good.append(line.split())
    X_good=[[float(j) for j in i] for i in X_good] 
        #del X_good[-1]
        #X_good=np.asarray(X_good)
    list0= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+layer+"_360.txt", "r")
    X_bad=[]
    for line in list0:
        X_bad.append(line.split())
    X_bad=[[float(j) for j in i] for i in X_bad]
    X_good.extend(X_bad)
    X=np.asarray(X_good)
    score=cross_val_score(clf, X, y, cv=cv)
    print score.mean()

In [ ]:
import numpy as np

layer=
list1 = open('/Users/manoj/Desktop/featuregood'+layer, "r")
X_good=[]
for line in list1 :
    X_good.append(line.split())
X_good=[[float(j) for j in i] for i in X_good]
del X_good[-1]
X_good=np.asarray(X_good)
X_good.shape

In [22]:
#six way classification 
gb='good'
layer_list=[]
clf = svm.SVC(kernel='linear', C=1)
#num_sample = 210
cv = ShuffleSplit(n_splits=10, test_size=0.1, random_state=0)
y_good=[1]*27+[2]*31+[3]*39+[4]*46+[5]*25+[6]*42
y_good=np.asarray(y_good)
y_bad=[1]*34+[2]*39+[3]*32+[4]*31+[5]*42+[6]*32
y_bad=np.asarray(y_bad)
if gb=='good':
    y=y_good
    for layer in layer_list:
        list0= open('/Users/manoj/good_bad_placecnn/features/'+gb+"360/"+gb+"_"+j+"_360.txt", "r")
        X_good=[]
        for line in list0:
            X_good.append(line.split())
        X_good=[[float(j) for j in i] for i in X_good]
        X_good=np.asarray(X_bad)
        X=X_good
        score_good=cross_val_score(clf, X, y, cv=cv)
        print layer
        print score_good.mean()
else:
    y=y_bad
    for layer in layer_list:
        list0= open('/Users/manoj/Desktop/'+gb+"/"+gb+"_"+j+".txt", "r")
        X_bad=[]
        for line in list0:
            X_bad.append(line.split())
        X_bad=[[float(j) for j in i] for i in X_bad]
        X_bad=np.asarray(X_bad)
        X=X_bad
        score_bad=cross_val_score(clf, X, y, cv=cv)
        print layer
        score_bad
        print score_bad.mean()
    


In [ ]:
(290400,)
(186624,)
(64896,)
(64896,)
(43264,)
(4096,)
(4096,)
(205,)